Access to a file and creation of embedding in a dataframe

In [1]:
file = open("stort.txt",'r')
text = file.readlines()
file.close

<function TextIOWrapper.close()>

In [2]:
def text_split(text):
    word=''
    sentence=''
    paragraph=[]
    for row in text:
        for char in row:
            if char !=' ':
                word=word+char
                #print(word)
            else:
                sentence=sentence+word
                #print(sentence)
                word=' '
            if len(sentence.split())==250:
                #print(sentence)
                paragraph.append(sentence)
                sentence=' '
    paragraph.append(sentence+word) 
    print(paragraph[-1])   
    return paragraph


In [3]:
paragraph = text_split(text)

  to create works that touched people's hearts. Our small studio in a quiet corner of town was filled with sketches, written pages, and photographs of our best moments.

Not everything was always easy. There were days when deadlines stressed us out, or when we fought over small things. But each time we found a way to come back to each other, stronger than before. Alessia had an extraordinary ability to see beyond difficulties, to find light even in the darkest moments. And it was that resilience, that ability to love without reservation, that made me fall in love with her more and more every day.

Do you want me to add other specific scenes, such as a journey, an important event, or a moment of deep reflection? I can expand the story further in many directions!




Create text embeddings

In [4]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Blank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def embedding_text(text):
    encoding = tokenizer.batch_encode_plus([text],
                padding="max_length",
                max_length=512,
                truncation=True,
                return_tensors='pt',
                add_special_tokens=True)
    input_ids = encoding['input_ids']
    return input_ids

for i in range(len(paragraph)):
    paragraph[i]=embedding_text(paragraph[i])
    print(paragraph[i])



tensor([[  101, 15669, 18719,  2050,  1010,  2026,  2293,  1024,  1037,  2466,
          1997,  2540,  1998,  6580, 15669, 18719,  2050,  1005,  1055,  2466,
          4269,  2006,  1037,  3500,  5027,  1010,  2043,  1996,  2422,  9478,
          3344,  1996,  6518,  1997,  9115, 20593,  1998,  1996,  3712,  2001,
          1037,  9428, 10866,  2630,  1012,  1996,  3628,  1010, 13426,  2075,
          1999,  2296,  8703,  1997,  2317,  1998,  5061,  1010,  7121,  2247,
          1996,  2380,  1005,  1055, 20859,  2066, 16074,  2015,  1997, 28855,
         15879,  5053,  1012,  1045,  5078,  1037,  2422,  4462,  6598,  2058,
          1037,  2422,  2630,  3797,  1010,  2007,  2601,  6312,  1998,  6247,
         28130,  1010,  1037,  6742,  2021,  2025,  3391, 11552,  3601,  1012,
         15669, 18719,  2050,  1010,  2006,  1996,  2060,  2192,  1010,  2246,
          2066,  2242,  2041,  1997,  2019,  8605,  2923,  4169,  1024,  2016,
          5078,  1037, 19351,  2140,  2665,  4377,  

In [7]:
user_question = 'alessia'
user_emb = embedding_text(user_question)
print(user_emb.size())
print(paragraph[0].size())

max_match=0
match_paragraph=[]
for ele in paragraph:
    print(cosine_similarity(user_emb,ele))
    if cosine_similarity(user_emb,ele) > max_match:
        
        max_match = cosine_similarity(user_emb,ele)
        match_paragraph=[ele,max_match]

contex = tokenizer.decode(match_paragraph[0][0], skip_special_tokens=True) 
print(contex)
print(match_paragraph[1])

torch.Size([1, 512])
torch.Size([1, 512])
[[0.20615429]]
[[0.03330526]]
[[0.01887738]]
[[0.0176914]]
[[0.01896126]]
[[0.02139148]]
[[0.03962744]]
[[0.06368445]]
alessia, my love : a story of heart and fate alessia's story begins on a spring afternoon, when the light breeze carried the scent of cherry blossom and the sky was a seemingly endless blue. the trees, blooming in every shade of white and pink, stretched along the park's avenues like sentinels of ethereal beauty. i wore a light grey jacket over a light blue shirt, with dark jeans and worn sneakers, a practical but not particularly elegant choice. alessia, on the other hand, looked like something out of an impressionist painting : she wore a pastel green dress that danced to the rhythm of the wind, and a white scarf that contrasted delicately with her long brown hair. on her feet were simple beige ballet shoes, which gave her an even more refined air. it was then that i saw her for the first time. alessia was sitting on a wooden

Model for text generation

In [8]:

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

input_text = f'based on this:\n {contex}.\n answer that:\n{user_question}'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(input_text)
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


based on this:
 alessia, my love : a story of heart and fate alessia's story begins on a spring afternoon, when the light breeze carried the scent of cherry blossom and the sky was a seemingly endless blue. the trees, blooming in every shade of white and pink, stretched along the park's avenues like sentinels of ethereal beauty. i wore a light grey jacket over a light blue shirt, with dark jeans and worn sneakers, a practical but not particularly elegant choice. alessia, on the other hand, looked like something out of an impressionist painting : she wore a pastel green dress that danced to the rhythm of the wind, and a white scarf that contrasted delicately with her long brown hair. on her feet were simple beige ballet shoes, which gave her an even more refined air. it was then that i saw her for the first time. alessia was sitting on a wooden bench faded by time, stroking a small white curly - haired dog that seemed to adore every moment spent with her. she had a sweet smile and an ab

c:\Users\Blank\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> is a story of love and fate</s>
